In [2]:
import pandas as pd

In [3]:
x1 = pd.read_csv('Run1.csv')
x2 = pd.read_csv('Run2.csv')
x3 = pd.read_csv('Run3.csv')
x4 = pd.read_csv('Run4.csv')
x5 = pd.read_csv('Run5.csv')
x6 = pd.read_csv('Run6.csv')

In [5]:
full_game_frame = pd.concat([x1, x2, x3, x4, x5, x6])
full_game_frame

,Team_Name,Points_Scored,Game_ID,Field Goals Made,Field Goals Attempted,Free Throws Made,Free Throws Attempted,Three Point Made,Three Point Attempted,Total Points,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Turnovers,Steals,Blocks,Fouls,Technical Fouls
0,Lac Courte Oreilles,57,33023420,8,32,4,20,1,3,21,11,24,36,3,24,4,2,0,0.0
1,Chequamegon,21,33023420,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,Lac Courte Oreilles,43,33023421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,Cornell,31,33023421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,Lac Courte Oreilles,33,33023423,26,67,8,10,6,25,66,14,23,37,20,10,16,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Appleton East,78,33027534,32,56,10,15,8,16,82,4,18,22,11,6,11,2,8,NaN
428,Marshfield,55,33027535,26,50,7,12,10,25,69,9,22,31,20,8,10,3,15,NaN
429,Appleton East,69,33027535,18,49,10,13,9,25,55,8,13,21,5,10,3,4,13,NaN
430,Neenah,59,33027536,0,0,0,0,0,0,55,0,0,0,0,0,0,0,0,NaN


In [11]:
full_game_frame.columns

Index(['Team_Name', 'Points_Scored', 'Game_ID', 'Field Goals Made',
       'Field Goals Attempted', 'Free Throws Made', 'Free Throws Attempted',
       'Three Point Made', 'Three Point Attempted', 'Total Points',
       'Offensive Rebounds', 'Defensive Rebounds', 'Total Rebounds', 'Assists',
       'Turnovers', 'Steals', 'Blocks', 'Fouls', 'Technical Fouls'],
      dtype='object')

In [27]:
def offensive_efficiency(row):
    if (row['Field Goals Attempted'] + row['Assists'] + row['Turnovers'] - row['Offensive Rebounds']) > 0:
        return ((row['Free Throws Made'] + row['Assists']) / (row['Field Goals Attempted'] + row['Assists'] + row['Turnovers'] - row['Offensive Rebounds']))
    else:
        return 0

# Efficient Points Scored function 
def efficient_points_scored(row):
   e_p_s = row['Offensive_Efficiency'] * row['Points_Scored']
   return e_p_s

# Raw EOP function 
def raw_EOP(row):
    return (.76 * (row['Assists'] + row['Points_Scored']) * row['Offensive_Efficiency'])

def get_ppa_two(row):
    total_points = 2 * (row['Field Goals Made'] - row['Three Point Made'])
    total_attempts = (row['Field Goals Attempted'] - row['Three Point Attempted'])
    if total_attempts > 0:
        return total_points / total_attempts
    else:
        return 0

def get_ppa_three(row):
    total_points = 3 * row['Three Point Made']
    total_attempts = row['Three Point Attempted']
    if total_attempts > 0:
        return total_points / total_attempts
    else:
        return 0

def get_total_ppa(row):
    total_points = 2 * (row['Field Goals Made'] - row['Three Point Made']) + 3 * row['Three Point Made']
    total_attempts = row['Field Goals Attempted']
    if total_attempts > 0:
        return total_points / total_attempts
    else:
        return 0

def team_scoring_poss(row):
    team_attempts = row['Field Goals Made']
    if row['Free Throws Attempted'] > 0:
        inner_part = (1 - (1 - (row['Free Throws Made'] / row['Free Throws Attempted']) ** 2))
    else:
        inner_part = 0
    return team_attempts + inner_part *  row['Free Throws Attempted'] * 0.4

def team_play_percent(row):
    numerator = row['Scoring_Possesions']
    denom = row['Field Goals Attempted'] + row['Free Throws Attempted'] * 0.4 + row['Turnovers']
    if denom > 0:
        return numerator / denom
    else:
        return 0

def possesions(row):
    fga = row['Field Goals Attempted']
    other_team_def_rebounds = (row['Field Goals Attempted'] - row['Field Goals Made']) - row['Offensive Rebounds']
    if (row['Offensive Rebounds'] + other_team_def_rebounds) > 0:
        reb_part = row['Offensive Rebounds']/(row['Offensive Rebounds'] + other_team_def_rebounds)
    else:
        reb_part = 0
    fg_part = row['Field Goals Attempted'] - row['Field Goals Made']
    return fga - reb_part * fg_part * 1.07 + row['Turnovers'] + row['Field Goals Attempted'] * 0.4

In [28]:
full_game_frame['Offensive_Efficiency'] = full_game_frame.apply(offensive_efficiency, axis = 1)
full_game_frame['Efficient Points Score'] = full_game_frame.apply(efficient_points_scored, axis = 1)
full_game_frame['Raw EOP'] = full_game_frame.apply(raw_EOP, axis = 1)
full_game_frame['Two Points Per Attempt'] = full_game_frame.apply(get_ppa_two, axis = 1)
full_game_frame['Three Points Per Attempt'] = full_game_frame.apply(get_ppa_three, axis = 1)
full_game_frame['Total Points Per Attempt'] = full_game_frame.apply(get_total_ppa, axis = 1)
full_game_frame['Scoring_Possesions'] = full_game_frame.apply(team_scoring_poss, axis = 1)
full_game_frame['Team Play Percent'] = full_game_frame.apply(team_play_percent, axis = 1)
full_game_frame['Possesions'] = full_game_frame.apply(possesions, axis = 1)
full_game_frame

,Team_Name,Points_Scored,Game_ID,Field Goals Made,Field Goals Attempted,Free Throws Made,Free Throws Attempted,Three Point Made,Three Point Attempted,Total Points,...,Technical Fouls,Offensive_Efficiency,Efficient Points Score,Raw EOP,Two Points Per Attempt,Three Points Per Attempt,Total Points Per Attempt,Scoring_Possesions,Team Play Percent,Possesions
0,Lac Courte Oreilles,57,33023420,8,32,4,20,1,3,21,...,0.0,0.145833,8.312500,6.650000,0.482759,1.00,0.531250,8.320000,0.130000,57.03
1,Chequamegon,21,33023420,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
2,Lac Courte Oreilles,43,33023421,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
3,Cornell,31,33023421,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
4,Lac Courte Oreilles,33,33023423,26,67,8,10,6,25,66,...,0.0,0.337349,11.132530,13.588434,0.952381,0.72,0.865672,28.560000,0.352593,88.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Appleton East,78,33027534,32,56,10,15,8,16,82,...,NaN,0.304348,23.739130,20.586087,1.200000,1.50,1.285714,34.666667,0.509804,80.12
428,Marshfield,55,33027535,26,50,7,12,10,25,69,...,NaN,0.391304,21.521739,22.304348,1.280000,1.20,1.240000,27.633333,0.440021,68.37
429,Appleton East,69,33027535,18,49,10,13,9,25,55,...,NaN,0.267857,18.482143,15.064286,0.750000,1.08,0.918367,21.076923,0.328301,70.04
430,Neenah,59,33027536,0,0,0,0,0,0,55,...,NaN,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00


In [29]:
full_games_joined = full_game_frame.merge(full_game_frame, on = 'Game_ID')

In [32]:
full_games_joined_2 = full_games_joined[full_games_joined['Team_Name_x'] != full_games_joined['Team_Name_y']]

In [43]:
full_games_joined_2 = full_games_joined_2[full_games_joined_2['Offensive_Efficiency_y'] > 0]
full_games_joined_2 = full_games_joined_2[full_games_joined_2['Offensive_Efficiency_x'] > 0]

In [45]:
full_games_joined_2                                           

,Team_Name_x,Points_Scored_x,Game_ID,Field Goals Made_x,Field Goals Attempted_x,Free Throws Made_x,Free Throws Attempted_x,Three Point Made_x,Three Point Attempted_x,Total Points_x,...,Technical Fouls_y,Offensive_Efficiency_y,Efficient Points Score_y,Raw EOP_y,Two Points Per Attempt_y,Three Points Per Attempt_y,Total Points Per Attempt_y,Scoring_Possesions_y,Team Play Percent_y,Possesions_y
37,Laconia,72,33023432,9,37,11,21,4,16,33,...,0.0,0.352941,11.647059,14.484706,0.976744,0.750000,0.887324,30.160000,0.354824,91.35
38,Campbellsport,33,33023432,28,71,9,15,7,28,72,...,0.0,0.301587,21.714286,18.336508,0.476190,0.750000,0.594595,11.304762,0.167726,69.52
45,Laconia,87,33023434,16,41,7,14,2,7,41,...,0.0,0.337500,13.837500,15.133500,1.200000,1.500000,1.322034,35.160000,0.475135,85.18
46,Lomira,41,33023434,33,59,9,15,12,24,87,...,0.0,0.205882,17.911765,14.708235,0.823529,0.857143,0.829268,17.400000,0.227154,76.70
49,Laconia,58,33023435,6,33,2,5,2,7,16,...,0.0,0.223684,3.578947,4.760000,0.727273,0.840000,0.768116,24.428571,0.310007,90.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11386,Appleton East,105,33027533,31,60,27,34,7,21,96,...,NaN,0.632353,60.705882,53.825882,1.687500,1.235294,1.530612,41.476923,0.541474,70.97
11389,Kimberly,82,33027534,29,55,8,8,12,24,78,...,NaN,0.304348,23.739130,20.586087,1.200000,1.500000,1.285714,34.666667,0.509804,80.12
11390,Appleton East,78,33027534,32,56,10,15,8,16,82,...,NaN,0.308642,25.308642,23.222222,1.096774,1.500000,1.272727,32.200000,0.445983,85.65
11393,Marshfield,55,33027535,26,50,7,12,10,25,69,...,NaN,0.267857,18.482143,15.064286,0.750000,1.080000,0.918367,21.076923,0.328301,70.04


In [50]:
full_games_joined_2.columns
for col in full_games_joined_2:
    if (col != 'Team_Name_x') & (col != 'Team_Name_y'):
        if '_x' in col:
            col_2 = col.replace('_x', '_Allowed')
            full_games_joined_2 = full_games_joined_2.rename(columns = {col : col_2})
        elif '_y' in col:
            col_3 = col.replace('_y', '')
            full_games_joined_2 = full_games_joined_2.rename(columns = {col : col_3})
final_games = full_games_joined_2.drop(columns = 'Team_Name_y')
final_games = final_games.rename(columns = {'Team_Name_x' : 'Team'})
final_games.to_csv('Basketbal_2021_2022_Stats.csv', index = False)